![Verne](https://www.vernegroup.com/wp-content/uploads/2020/07/LOGO-VERNE-TECHNOLOGY-GROUP-3.png)
# Interpretabilidad de Modelos de Machine Learning con Interpret 

## ¿Por qué necesitamos Interpretar Modelos?

Hay quien dice que si los resultados son correctos, ¿qué importa como se llegue a ellos? Olvidando que como humanos que somos la lógica y el razonamiento va siempre de la mano de nuestras decisiones. Además, si tratamos nuestros modelos como cajas negras, será complicado detectar posibles problemas, tales como los sesgos o el sobreentrenamiento. 

Como científicos de datos, necesitamos entender el por qué un determinado modelo arroja un resultado. Además, necesitamos poder explicar al usuario final por qué toma el modelo sus decisiones. A mayores de todo esto, debemos de hacer un ML responsable que evite sesgos de cualquier tipo. Por todo ello, se hace indispensable la interpretabilidad. Podríamos resumir las necesidades en:
- Legales. GDPR incorpora determinados requisitos
- Morales. Para evitar sesgos y asegurarnos que el modelo toma decisiones justas.
- De Negocio. Saber interpretar por qué el modelo toma sus decisiones y que características del proceso son las que tienen mayor relevancia. 

## ¿Por qué Interpret?
Es un paquete que encapsula muchas de las técnicas actuales de interpretabilidad de modelos de ML. Interpret clasifa los modelos en:
- Glassbox. Son modelos que parten de algoritmos diseñados para ser interpretables, como los Árboles de Decisión o los modelos Lineales, que habitualmente proporcionan predicciones exactas
- Blackbox. Son modelos cuya interpretabilidad no es trivial, y es necesario utilizar determinadas técnicas para extraer esa información. 

En este ejemplo nos centraremos en un modelo blackbox, que será un Clasificador de RandonForest. Utilizaremos uno de los conjuntos de datos más utilizados en ML para demostrar técnicas de clasificación, que son los datos de Censo de adultos del año 1994, para intentar predecir si los ingresos de una persona son superiores a 50k USD o no. 

## El código

### Carga de Datos y Definición del problema
vamos a utilizar un conjunto de datos muy utilizado en el mundo de ML cuando se quieren explicar, fundamentalmente, problemas de clasificación, que es el conjunto de datos de censo. 


Descripción del conjunto de datos: https://archive.ics.uci.edu/ml/datasets/Adult

El conjunto de datos lo tenemos disponible en "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

Este conjunto de datos se utiliza para demostrar varios problemas sencillos de ML, en nuestro caso, intentaremos predecir, según otros datos demográficos, si el salario de una persona estaría por encima o por debajo de los 50k

### Instalación y carga de paquetes

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
#Cargar Datos
# Descripción del conjunto de datos: https://archive.ics.uci.edu/ml/datasets/Adult

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]
train_cols = df.columns[0:-1] #quitamos la columna Income
label = df.columns[-1] #Asignamos income como etiqueta
X = df[train_cols] 
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #convertimos en 0 o 1 si es >50k

In [4]:
# Transformamos las variables categóricas
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)

seed = 1  
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)


### Creamos el pipeline y entrenamos el Clasificador

In [5]:

#Entrenamos el clasificador
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

#Un sistema Blackbox puede incluir preprocesado, no solo el clasificador

pca = PCA()
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

blackbox_model = Pipeline([('pca', pca), ('rf', rf)])
blackbox_model.fit(X_train, y_train)



Pipeline(steps=[('pca', PCA()), ('rf', RandomForestClassifier(n_jobs=-1))])

### ¿Cómo de bueno es el modelo? Rendimiento

In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = blackbox_model.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[4652  374]
 [ 615  872]]
              precision    recall  f1-score   support

           0       0.88      0.93      0.90      5026
           1       0.70      0.59      0.64      1487

    accuracy                           0.85      6513
   macro avg       0.79      0.76      0.77      6513
weighted avg       0.84      0.85      0.84      6513



In [ ]:
#Instalando el paquete interpret
#!pip install interpret

In [7]:
from interpret import show
from interpret.perf import ROC

blackbox_perf = ROC(blackbox_model.predict_proba).explain_perf(X_test, y_test, name='Modelo de Caja Negra')
show(blackbox_perf)

c:\Users\asoto\interpret-env\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\asoto\interpret-env\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\asoto\interpret-env\lib\site-packages\interpret\visual\udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
c:\Users\asoto\interpret-env\lib\site-packages\interpret\visual\udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
c:\Users\asoto\interpret-env\lib\site-packages\interpret\visual\udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replac

<!-- http://127.0.0.1:7001/2445544095360/ -->

### Interpretación Local con LIME
La idea es bastante intuitiva. Primero, olvídate de los datos de entrenamiento e imagina que solo tienes el modelo de caja negra donde puedes ingresar puntos de datos y obtener las predicciones del modelo. Puede sondear la caja con la frecuencia que desee. Su objetivo es entender por qué el modelo de aprendizaje automático hizo una cierta predicción. LIME prueba lo que sucede con las predicciones cuando se proporcionan variaciones de los datos en el modelo de aprendizaje automático. LIME genera un nuevo conjunto de datos que consta de muestras persilenciadas y las predicciones correspondientes del modelo de caja negra. En este nuevo DataSet LIME, a continuación, entrena un modelo interpretables, que se pondera por la proximidad de las instancias muestreadas a la instancia de interés. El modelo interpretables puede ser cualquier cosa del capítulo de modelos interpretables, por ejemplo Lasso o un árbol de decisión. El modelo aprendido debe ser una buena aproximación de las predicciones del modelo de aprendizaje automático localmente, pero no tiene que ser una buena aproximación global. Este tipo de precisión también se denomina fidelidad local.


In [8]:
from interpret.blackbox import LimeTabular
from interpret import show

#Los "explicadores" de cajas negras necesitan una función de predicción, y opcionalmente un conjunto de datos
lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train)

#Seleccionamos las instancias a explicar, y podemos pasar opcionalmente las etiquetas si las tenemos
lime_local = lime.explain_local(X_test[:5], y_test[:5], name='LIME')

show(lime_local)

c:\Users\asoto\interpret-env\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but PCA was fitted with feature names

c:\Users\asoto\interpret-env\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but PCA was fitted with feature names

c:\Users\asoto\interpret-env\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but PCA was fitted with feature names

c:\Users\asoto\interpret-env\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but PCA was fitted with feature names

c:\Users\asoto\interpret-env\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but PCA was fitted with feature names

c:\Users\asoto\interpret-env\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but PCA was fitted with feature names

c:\Users\asoto\interpret-env\lib\site-packages\sklea

<!-- http://127.0.0.1:7001/2445601906528/ -->

In [9]:
from interpret.blackbox import ShapKernel
import numpy as np

background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=background_val, feature_names=feature_names)
shap_local = shap.explain_local(X_test[:5], y_test[:5], name='SHAP')
show(shap_local)

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
  0%|          | 0/5 [00:00<?, ?it/s]X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
 20%|██        | 1/5 [00:00<00:00,  9.58it/s]X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
 60%|██████    | 3/5 [00:00<00:00, 13.87it/s]X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was

<!-- http://127.0.0.1:7001/2445602269184/ -->

### Intepretabilidad Global con Sensibilidad de Morris

El método comienza muestreando un conjunto de valores iniciales dentro de los rangos definidos de valores posibles para todas las variables de entrada y calculando el resultado del modelo. El segundo paso cambia los valores de una variable (todas las demás entradas restantes en sus valores iniciales) y calcula el cambio resultante en el resultado del modelo en comparación con la primera ejecución. A continuación, se cambian los valores de otra variable (la variable anterior se mantiene en su valor modificado y todas las demás se mantienen en sus valores iniciales) y se calcula el cambio resultante en el resultado del modelo en comparación con la segunda ejecución. Esto continúa hasta que se cambian todas las variables de entrada. Este procedimiento se repite r veces (donde se toman generalmente entre 5 y 15), cada vez con un conjunto diferente de valores de inicio, lo que conduce a un número de r(k + 1) corridas, donde k es el número de variables de entrada.

In [10]:

from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=blackbox_model.predict_proba, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Sensibilidad de Morris")

show(sensitivity_global)

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names


<!-- http://127.0.0.1:7001/2445602281920/ -->

### Intepretabilidad Global con PDP
La gráfica de dependencia parcial (gráfica de PDP o PD corta) muestra el efecto marginal que una o dos características tienen en el resultado pronosticado de un modelo de aprendizaje automático (j. h. Friedman 200127). Una gráfica de dependencia parcial puede mostrar si la relación entre el destino y una entidad es lineal, monótona o más compleja. Por ejemplo, cuando se aplica a un modelo de regresión lineal, los trazados de dependencia parcial siempre muestran una relación lineal.


In [11]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global(name='Dependencias Parciales')

show(pdp_global)

X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid feature names, but PCA was fitted with feature names
X does not have valid fea

<!-- http://127.0.0.1:7001/2445602267600/ -->

## Algoritmos específicos de InterpretML: ExplainableBoostingClassifier

In [12]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier(random_state=seed)
ebm.fit(X_train, y_train)

ExplainableBoostingClassifier(feature_names=['Age', 'fnlwgt', 'EducationNum',
                                             'CapitalGain', 'CapitalLoss',
                                             'HoursPerWeek', 'WorkClass. ?',
                                             'WorkClass. Federal-gov',
                                             'WorkClass. Local-gov',
                                             'WorkClass. Never-worked',
                                             'WorkClass. Private',
                                             'WorkClass. Self-emp-inc',
                                             'WorkClass. Self-emp-not-inc',
                                             'WorkClass. State-gov',
                                             'WorkClass. Without-pay',
                                             'Education. 10th',
                                             'Education...
                                             'categorical', 'categorical',
     

In [17]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

In [18]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist)

In [15]:
ebm_global = ebm.explain_global()
show(ebm_global)

In [16]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5])
show(ebm_local)

In [ ]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

## Otros Algoritmos: ClassificationTree

In [19]:
#Otros Modelos

from interpret.glassbox import ClassificationTree


tree = ClassificationTree()
tree.fit(X_train, y_train)

tree_perf = ROC(tree.predict_proba).explain_perf(X_test, y_test, name='Classification Tree')

In [20]:
show(tree_perf)


In [21]:

tree_global = tree.explain_global(name='Classification Tree')

show(tree_global)
